Importanción de librerias

In [57]:
import warnings
warnings.filterwarnings("ignore")

In [58]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

### Parte 1: Procesamiento de los datos

Cargando de los datos

In [59]:
dataset = pd.read_csv('datos_crudos/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Separando las variables independientes y la variable dependiente

In [60]:
X = dataset.iloc[:, 3:13] #Variables independientes
y = dataset.iloc[:, 13] #Variable dependiente

Codificación de datos categóricos

In [61]:
#Codificando la variable categórica "Geography"
one_hot_encoding = pd.get_dummies(X['Geography'], prefix='Geography').astype(int)
X = pd.concat([X, one_hot_encoding], axis=1)
#Se elimina la columna "Geography_Spain" para evitar problemas de multicolinealidad
X.drop(['Geography', 'Geography_Spain'], axis=1, inplace=True)
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany
0,619,Female,42,2,0.00,1,1,1,101348.88,1,0
1,608,Female,41,1,83807.86,1,0,1,112542.58,0,0
2,502,Female,42,8,159660.80,3,1,0,113931.57,1,0
3,699,Female,39,1,0.00,2,0,0,93826.63,1,0
4,850,Female,43,2,125510.82,1,1,1,79084.10,0,0


In [62]:
#Codificando la variable categórica "Gender"
sexo = pd.get_dummies(X['Gender'], prefix='sex').astype(int)
X = pd.concat([X, sexo], axis=1)
X.drop(['Gender', 'sex_Male'], axis=1, inplace=True)
X.rename(columns={'sex_Female':'Gender'}, inplace=True)
X.head()
#La columna Gender 1 es para femenino y 0 para masculino

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Gender
0,619,42,2,0.00,1,1,1,101348.88,1,0,1
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,1
3,699,39,1,0.00,2,0,0,93826.63,1,0,1
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1


Dividisión del dataframe en entrenamiento y en prueba

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Escalando las variables

In [64]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Parte 2: Construcción de la red neuronal artificial (RNA)

Inicializando la RNA

In [65]:
#Se crea la RNA
clasificador = Sequential()

#Se agregan las capas ocultas
clasificador.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))
clasificador.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

#Se agrega la capa de salidas
clasificador.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

#Compilar la RNA
clasificador.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics = ["accuracy"]
)

Entrenando la RNA

In [66]:
clasificador.fit(
    x=X_train_scaled,
    y=y_train,
    batch_size=10,
    epochs=50
)

Epoch 1/50


800/800 [==============================] - 6s 5ms/step - loss: 0.4855 - accuracy: 0.7956
Epoch 2/50
800/800 [==============================] - 3s 3ms/step - loss: 0.4279 - accuracy: 0.7960
Epoch 3/50
800/800 [==============================] - 3s 3ms/step - loss: 0.4225 - accuracy: 0.7960
Epoch 4/50
800/800 [==============================] - 2s 3ms/step - loss: 0.4185 - accuracy: 0.8175
Epoch 5/50
800/800 [==============================] - 3s 3ms/step - loss: 0.4159 - accuracy: 0.8276
Epoch 6/50
800/800 [==============================] - 3s 3ms/step - loss: 0.4139 - accuracy: 0.8307
Epoch 7/50
800/800 [==============================] - 3s 3ms/step - loss: 0.4125 - accuracy: 0.8304
Epoch 8/50
800/800 [==============================] - 3s 3ms/step - loss: 0.4108 - accuracy: 0.8335
Epoch 9/50
800/800 [==============================] - 3s 3ms/step - loss: 0.4107 - accuracy: 0.8325
Epoch 10/50
800/800 [==============================] - 3s 3ms/step - loss: 0.4095 - accuracy: 0.8330
Epoch 11/5

### Parte 3: Evaluación de la RNA

Se realiza las predicciones

In [67]:
y_pred = clasificador.predict(x=X_test_scaled)
y_pred = (y_pred > 0.5)

63/63 [==============================] - 0s 2ms/step


Se crea la matriz de confusión

In [68]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1546,   49],
       [ 267,  138]], dtype=int64)

### Parte 4: Predecir una nueva observación

Utilizando el modelo de RNA para predecir si el cliente con la siguiente información abandonará el banco:

- Geografia: Francia

- Puntaje de crédito: 600

- Género masculino

- Edad: 40 años de edad

- Tenencia: 3 años.

- Saldo: $ 60000

- Número de productos: 2

- ¿Este cliente tiene una tarjeta de crédito? Sí

- ¿Es este cliente un miembro activo? Sí

- Salario estimado: $ 50000

In [69]:
obs = np.array([[600, 40, 3, 60000, 2, 1, 1, 50000, 1, 0, 0]])
obs_scaled = scaler.transform(obs)
nueva_pred = clasificador.predict(obs_scaled)
print(nueva_pred > 0.5)

1/1 [==============================] - 0s 57ms/step
[[False]]


### Parte 5: Validación cruzada